In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
from LumFunc_miniJPAS import LumFunc_hist
from scipy.integrate import simpson

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_7.csv', sep=',', header=1,
    converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0] # Drop bad rows due to bad query
cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
cat = cat.reset_index()

pm_flx = np.stack(cat['FLUX_AUTO'].to_numpy()).T * 1e-19
pm_err = np.stack(cat['FLUX_RELERR_AUTO'].to_numpy()).T * pm_flx

In [ ]:
mag = flux_to_mag(pm_flx, np.array(w_central.reshape(-1, 1)))

In [ ]:
plt.hist(mag[-2], bins=np.linspace(15, 26, 30))
plt.show()

In [ ]:
w_lya = 1215.67 # A

N_sources = len(cat['FLUX_AUTO'])
z_nb_Arr = w_central[:-4] / w_lya - 1
print(N_sources)

In [ ]:
pm_mask = mask_proper_motion(cat)
mag_mask = mag[-2] > 17

mask = pm_mask & mag_mask

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 30, mask=mask)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    400, obs=True, mask=mask)
other_lines = identify_lines(line_other, pm_flx, pm_err)

In [ ]:
mag_min = 17
mag_max = 30

mag_cut = (mag[-2] > mag_min) & (mag[-2] < mag_max)

z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

##
nb_min = 5
nb_max = 25

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5)/ w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5)/ w_lya - 1

z_cut = (z_min < z_Arr) & (z_Arr < z_max)

mask = z_cut & mag_cut

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_other, z_Arr, mask=mask
)

In [ ]:
len(np.where(nice_lya)[0])

In [ ]:
for k, src in enumerate(np.where(nice_lya)[0]):
    if k == 5: break
    fig = plt.figure(figsize=(8, 6))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    print(f'z_NB = {z_Arr[src]}')
    
    ax.plot(w_central[:56], cont_est_lya[:, src], ls='--', c='black')

    ax.axvline(w_central[lya_lines[src]], label='Selected NB')
    ax.legend(fontsize=13)
    plt.show()

In [ ]:
N_bins = 20
bins = np.linspace(42.5, 45.25, N_bins + 1)

_, _, L_Arr, L_e_Arr = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, nice_lya
)
L_Arr -= 0.04206043155753747


total_hist, b = np.histogram(L_Arr[nice_lya], bins=bins)

LF_bins = [(b[i] + b[i + 1]) / 2 for i in range(len(b) - 1)]
bin_width = b[1] - b[0]
volume = z_volume(z_min, z_max, 0.895)

In [ ]:
correct = np.array([ 0.        ,  0.        ,  0.        , 38.90654585,  9.08544692,
        6.9789645 ,  2.21477827,  1.24907731,  0.63762022,  0.70178861,
        0.80105365,  0.8532261 ,  1.13501275,  1.77083291,  3.20890848,
        4.80531436,  6.27717816,  7.13079086,  8.78795319, 11.30359997])

In [ ]:
## Let's compute the error contributio due to L uncertainty as in Spinoso2020:
# Perturbing the L with its error and computing 1000 Luminosity functions with
# the result.

hist_i_mat = np.zeros((1000, N_bins))
for k in range(1000):
    L_perturbed = np.log10(10 ** L_Arr + L_e_Arr * np.random.randn(N_sources))

    hist_i_mat[k], _ = np.histogram(L_perturbed[nice_lya], bins=bins)
hist_i_mat *= correct

L_LF_err_percentiles = np.percentile(hist_i_mat, [16, 50, 84], axis=0)
L_LF_err_plus = L_LF_err_percentiles[2] - L_LF_err_percentiles[1]
L_LF_err_minus = L_LF_err_percentiles[1] - L_LF_err_percentiles[0]
hist_median = L_LF_err_percentiles[1]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

yerr_cor_plus = ((total_hist * correct) ** 0.5 + L_LF_err_plus)\
     / volume / bin_width
yerr_cor_minus = ((total_hist * correct) ** 0.5 + L_LF_err_minus)\
     / volume / bin_width

ax.errorbar(LF_bins, total_hist / volume / bin_width,
    yerr= total_hist ** 0.5 / volume / bin_width,
    marker='.', linestyle='', markersize=15, label='Uncorrected')
ax.errorbar(LF_bins, total_hist / volume / bin_width * correct,
    yerr= [yerr_cor_minus, yerr_cor_plus],
    marker='.', linestyle='', markersize=15, label='Corrected')
# ax.errorbar(LF_bins, hist_median / volume / bin_width * correct,
#     yerr= [yerr_cor_minus, yerr_cor_plus],
#     marker='.', linestyle='', markersize=15, label='Corrected median histogram')

Lx = np.linspace(10 ** 42, 10 ** 46, 10000)
phistar1 = 3.33e-6
Lstar1 = 10 ** 44.65
alpha1 = -1.35
Phi1 = schechter(Lx, phistar1, Lstar1, alpha1) * Lx * np.log(10)

phistar2 = 10 ** -3.45
Lstar2 = 10 ** 42.93
alpha2 = -1.93
Phi2 = schechter(Lx, phistar2, Lstar2, alpha2) * Lx * np.log(10)

ax.plot(np.log10(Lx), Phi1, label='Spinoso 2020 (QSO)')
ax.plot(np.log10(Lx), Phi2, label='Sobral 2018 (SF)')

ax.set_yscale('log')
ax.set_xlabel(r'$L_{\mathrm{Ly}\alpha}$ (erg s$^{-1}$)', fontsize=15)
ax.set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)',
    fontsize=15)
ax.set_ylim(1e-8, 5e-3)
ax.set_xlim(42, 45.5)
ax.legend(fontsize=12)

plt.show()

In [ ]:
vol = z_volume(2.45, 3.54, 0.895)

L_integrate = np.linspace(10 ** 43.5, 10 ** 44.55, 1000)
Phi_integrate = schechter(L_integrate, phistar1, Lstar1, alpha1)

L_integrate2 = 10 ** np.linspace(43, 43.4, 1000)
Phi_integrate2 = schechter(L_integrate2, phistar1, Lstar1, alpha1)

Nt = simpson(Phi_integrate, L_integrate) * vol
Ne = Nt - simpson(Phi_integrate2, L_integrate2) * vol

print('N_expected = {0:0.0f} +- {1:0.0f}'.format(Nt, Ne))